In [543]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Portfolio Analysis

## Objective: To understand how turnover affects profit in Las Vegas from 2015 - 2022

### Facts:
#### Bright Leaf
 - Purchase 1/2013
 - Purchase Price: 109k
 - Assumed value: $395,900 - from Zillow
 - Rent raised between 3-5% each year
 - 1 turn over in 2019 but new tenant was old tenant's sister, so rent was not pushed to market rent
 
#### Fort Pike
 - Purchase 2/2013
 - Purchase Price: 187k
 - Assumed value: $527,500 - from Zillow
 - Rent raised between 3-5% each year if tenant renewed
 - 3 turn overs in the period of 2015-2022 and each time rent was listed for market rent (2018, 2020, 2021)

#### How does a move out affect profit?
 - Each move out requires cleaning and repairing the home for a new tenant
 - More expenses incurr because the time it takes to repair home and get it rerented out takes about a month
 - Also have to pay administrative fees to property management to find new tenants
 - New tenants usually have a lot of maintenance request the first 6 months of tenancy because they find broken things that the old tenant lived with

In [544]:
data = pd.read_csv("data.csv")

In [545]:
data.head()

,Unnamed: 0,Type,Date,Num,Name,Memo,Clr,Split,Amount,Balance
0,CHASE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Check,12/17/14,NaN,FRANCHISE TAX BOARD,2014 chase chk #444,X,LLC Tax,-800.0,-800.0
2,NaN,Deposit,12/30/14,NaN,"GoldenWest Management, Inc",1828 BRIGHT LEAF 2014,X,Rental Income,12540.0,11740.0
3,NaN,Check,12/30/14,NaN,"GoldenWest Management, Inc",BRIGHT LEAF 2014,X,Management Fees,-900.0,10840.0
4,NaN,Check,12/30/14,NaN,State Farm Fire and Casualty Company,BRIGHT LEAF 2014,X,Insurance Expense,-532.0,10308.0


In [546]:
data["Date"] = pd.to_datetime(data["Date"])

In [547]:
data["Year"] = (data["Date"].apply(lambda x: x.year))

In [548]:
df = data.loc[data["Year"].isin(np.arange(2015, 2022+1))]
df = df.loc[df["Type"].isin(["Deposit", "Check"])]

In [549]:
df["is_bright_leaf"] = df["Memo"].str.lower().str.contains("bright|leaf")

In [550]:
df["is_fort_pike"] = df["Memo"].str.lower().str.contains("fort|pike")

In [551]:
def selector(row):
    if row["is_bright_leaf"]:
        return "bright leaf"
    elif row["is_fort_pike"]:
        return "fort pike"
    else:
        return None

In [552]:
df["Property"] = df.apply(selector, axis=1)

In [571]:
#Looking for outlier expenses
outlier = df.loc[(df["Amount"]<-3000) & (df["Split"]=="Repairs and Maintenance")].index
df = df.drop(outlier)

In [572]:
df.drop(["Unnamed: 0", "Date", "Num", "Clr", "Balance", "Memo", "is_bright_leaf", "is_fort_pike"], axis=1, inplace=True)

### Percent Growth of each property Does percent growth of house really matter?

In [573]:
def perct_growth(row):
    return (row["Assumed Value"] - row["Purchase Price"])/row["Purchase Price"]


prop_data = {
    "Property":["Fort Pike", "Bright Leaf"],
    "Purchase Price": [187000, 109000],
    "Assumed Value": [527500, 395900]
}

prop = pd.DataFrame(data = prop_data)
prop["Pct Growth"] = prop.apply(perct_growth, axis=1)*100
prop

,Property,Purchase Price,Assumed Value,Pct Growth
0,Fort Pike,187000,527500,182.085561
1,Bright Leaf,109000,395900,263.211009


In [574]:
fig = px.bar(prop, x="Property", y="Pct Growth", color="Property", title="Percent Growth of Property")
fig.show()

In [542]:
write_file("PCT-Growth.html", fig.to_html())

### Comparing revenue of each property from 2015-2021

In [575]:
income = df.loc[df["Split"]=="Rental Income"].groupby(["Property", "Year"]).sum().reset_index()

In [576]:
income

,Property,Year,Amount
0,bright leaf,2015.0,12800.00
1,bright leaf,2016.0,12975.00
2,bright leaf,2017.0,12595.00
3,bright leaf,2018.0,13140.00
4,bright leaf,2019.0,11280.00
5,bright leaf,2020.0,14304.20
6,bright leaf,2021.0,14940.00
7,bright leaf,2022.0,10360.00
8,fort pike,2015.0,18910.00
9,fort pike,2016.0,18545.00


In [577]:
fig = px.line(income, x="Year", y="Amount", color="Property", title="Revenue")

In [578]:
fig

In [579]:
write_file("Revenue.html", fig.to_html())

### Looking at Repairs and Maintenance as a proportion of Revenue

In [580]:
#Fort Pike
fp = df.loc[df["Property"]=="fort pike"]

fp_rev = fp.loc[fp["Split"]=="Rental Income"]
fp_rev = fp_rev.groupby("Year").sum().reset_index()

fp_rm = fp.loc[fp["Split"]=="Repairs and Maintenance"]
fp_rm = fp_rm.groupby("Year").sum().reset_index()
fp_rm["Amount"]= fp_rm["Amount"]*-1
print(fp_rm)

#Bright Leaf
bl = df.loc[df["Property"]=="bright leaf"]

bl_rev = bl.loc[bl["Split"]=="Rental Income"]
bl_rev = bl_rev.groupby("Year").sum().reset_index()

bl_rm = bl.loc[bl["Split"]=="Repairs and Maintenance"]
bl_rm = bl_rm.groupby("Year").sum().reset_index()
bl_rm["Amount"]= bl_rm["Amount"]*-1
bl_rm

     Year   Amount
0  2015.0   360.00
1  2016.0   704.00
2  2017.0   240.00
3  2018.0  1550.50
4  2019.0   200.00
5  2020.0  3029.27
6  2021.0  4776.70
7  2022.0  3798.22


,Year,Amount
0,2015.0,657.00
1,2016.0,280.00
2,2017.0,2151.23
3,2018.0,300.00
4,2019.0,135.00
5,2020.0,1040.00
6,2021.0,85.00
7,2022.0,120.00


In [581]:
fig = make_subplots(
    rows=1, cols=2, subplot_titles=("Fort Pike", "Bright Leaf")
)
#Fort Pike Graph
trace1 = go.Bar(x = fp_rev["Year"], y=fp_rev["Amount"], marker=dict(color="green", opacity=1), name="Revenue - Fort Pike")
trace2 = go.Bar(x = fp_rm["Year"], y=fp_rm["Amount"], marker=dict(color="red", opacity=1), name="Repairs and Maintenance - Fort Pike")
fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=1, col=1)
fig.update_xaxes(title_text="Year", row=1, col=1)
fig.update_yaxes(title_text="Amount",range=[0, 22000], row=1, col=1)

#Bright Leaf Graph
trace3 = go.Bar(x = bl_rev["Year"], y=bl_rev["Amount"], marker=dict(color="green", opacity=1), name="Revenue - Bright Leaf")
trace4 = go.Bar(x = bl_rm["Year"], y=bl_rm["Amount"], marker=dict(color="red", opacity=1), name="Repairs and Maintenance - Bright Leaf")
fig.add_trace(trace3, row=1, col=2)
fig.add_trace(trace4, row=1, col=2)
fig.update_xaxes(title_text="Year", row=1, col=2)
fig.update_yaxes(title_text="Amount", range=[0, 22000], row=1, col=2)

fig.update_layout(title = "Repairs and Maintenace as a Proportion of Revenue", barmode="overlay")

In [582]:
write_file("Repairs and Maintenance Rev.html", fig.to_html())

### Look at Net Operating Income (Revenue - Total Operating Expenses)

In [583]:
goi_fp = fp.loc[fp["Split"]=="Rental Income"].groupby("Year")["Amount"].sum() #Gross Operating Income
tox_fp = fp.loc[fp["Split"]!="Rental Income"].groupby("Year")["Amount"].sum()*-1 # Total Operating Expenses
noi_fp = (goi_fp - tox_fp).reset_index()
noi_fp

,Year,Amount
0,2015.0,13607.18
1,2016.0,12849.92
2,2017.0,13452.89
3,2018.0,10203.81
4,2019.0,15145.52
5,2020.0,10837.02
6,2021.0,11216.79
7,2022.0,8593.19


In [584]:
goi_bl = bl.loc[bl["Split"]=="Rental Income"].groupby("Year")["Amount"].sum() #Gross Operating Income
tox_bl = bl.loc[bl["Split"]!="Rental Income"].groupby("Year")["Amount"].sum()*-1 # Total Operating Expenses
noi_bl = (goi_bl - tox_bl).reset_index()
noi_bl

,Year,Amount
0,2015.0,9182.31
1,2016.0,9594.04
2,2017.0,6906.62
3,2018.0,9605.15
4,2019.0,7776.90
5,2020.0,9515.75
6,2021.0,12453.00
7,2022.0,7401.30


In [585]:
noi_fp = (goi_fp - tox_fp).reset_index().rename(columns = {"Amount":"Amount_FP"})
noi_fp

,Year,Amount_FP
0,2015.0,13607.18
1,2016.0,12849.92
2,2017.0,13452.89
3,2018.0,10203.81
4,2019.0,15145.52
5,2020.0,10837.02
6,2021.0,11216.79
7,2022.0,8593.19


In [586]:
noi_bl = (goi_bl - tox_bl).reset_index().rename(columns = {"Amount":"Amount_BL"})
noi_bl

,Year,Amount_BL
0,2015.0,9182.31
1,2016.0,9594.04
2,2017.0,6906.62
3,2018.0,9605.15
4,2019.0,7776.90
5,2020.0,9515.75
6,2021.0,12453.00
7,2022.0,7401.30


In [587]:
fig = make_subplots(
    rows=1, cols=1)
#Fort Pike Graph
trace1 = go.Bar(x = noi_fp["Year"], y=noi_fp["Amount_FP"], marker=dict(color="Blue", opacity=1), name="Fort Pike")
fig.add_trace(trace1)

#Bright Leaf Graph
trace2 = go.Bar(x = noi_bl["Year"], y=noi_bl["Amount_BL"], marker=dict(color="green", opacity=1), name="Bright Leaf")
fig.add_trace(trace2)

fig.update_xaxes(title_text="Year")
fig.update_yaxes(title_text="Amount")
fig.update_layout(title = "Net Operating Income", barmode="group")

In [588]:
write_file("NOI.html", fig.to_html())

### Looking at Cap Rates (Net Operating Income / Market Value of Property)

In [589]:
#September Zestimate for each year
zestimate_fp = {
    "Year":[year for year in range(2015,2022+1)],
    "Zestimate_FP":[304200, 300400, 334600, 347100, 365300, 385800, 480400, 527500]
}
assumed_value_fp = pd.DataFrame(data=zestimate_fp)

In [590]:
#September Zestimate for each year
zestimate_bl = {
    "Year":[year for year in range(2015,2022+1)],
    "Zestimate_BL":[143300, 193400, 213300, 264600, 264100, 285600, 353300, 395900]
}
assumed_value_bl = pd.DataFrame(data=zestimate_bl)

In [591]:
#Changing the Year from float to int
noi_fp["Year"] = noi_fp["Year"].astype("int")
noi_bl["Year"] = noi_bl["Year"].astype("int")

In [592]:
cap_fp = noi_fp.merge(assumed_value_fp, on="Year", how="inner")
cap_fp["Cap_Rate_FP"] = (cap_fp["Amount_FP"]/cap_fp["Zestimate_FP"])*100
cap_fp

,Year,Amount_FP,Zestimate_FP,Cap_Rate_FP
0,2015,13607.18,304200,4.473103
1,2016,12849.92,300400,4.277603
2,2017,13452.89,334600,4.020589
3,2018,10203.81,347100,2.939732
4,2019,15145.52,365300,4.146050
5,2020,10837.02,385800,2.808974
6,2021,11216.79,480400,2.334886
7,2022,8593.19,527500,1.629041


In [593]:
cap_bl = noi_bl.merge(assumed_value_bl, on="Year", how="inner")
cap_bl["Cap_Rate_BL"] = (cap_bl["Amount_BL"]/cap_bl["Zestimate_BL"])*100
cap_bl

,Year,Amount_BL,Zestimate_BL,Cap_Rate_BL
0,2015,9182.31,143300,6.407753
1,2016,9594.04,193400,4.960724
2,2017,6906.62,213300,3.237984
3,2018,9605.15,264600,3.630064
4,2019,7776.90,264100,2.944680
5,2020,9515.75,285600,3.331845
6,2021,12453.00,353300,3.524766
7,2022,7401.30,395900,1.869487


In [594]:
#Cap Rate with no expenses Fort Pike
goi_fp_ = goi_fp.reset_index() 
cap_fp_ = goi_fp_.merge(assumed_value_fp, on="Year", how="inner")
cap_fp_["Cap_Rate_FP"] = (cap_fp_["Amount"]/cap_fp_["Zestimate_FP"])*100
cap_fp_

,Year,Amount,Zestimate_FP,Cap_Rate_FP
0,2015.0,18910.00,304200,6.216305
1,2016.0,18545.00,300400,6.173435
2,2017.0,18705.00,334600,5.590257
3,2018.0,16770.00,347100,4.831461
4,2019.0,20530.00,365300,5.620038
5,2020.0,21115.00,385800,5.473043
6,2021.0,20662.16,480400,4.301032
7,2022.0,17472.00,527500,3.312227


In [595]:
#Cap Rate with no expenses Bright Leaf
goi_bl_ = goi_bl.reset_index()
cap_bl_ = goi_bl_.merge(assumed_value_bl, on="Year", how="inner")
cap_bl_["Cap_Rate_BL"] = (cap_bl_["Amount"]/cap_bl_["Zestimate_BL"])*100
cap_bl_

,Year,Amount,Zestimate_BL,Cap_Rate_BL
0,2015.0,12800.0,143300,8.932310
1,2016.0,12975.0,193400,6.708893
2,2017.0,12595.0,213300,5.904829
3,2018.0,13140.0,264600,4.965986
4,2019.0,11280.0,264100,4.271109
5,2020.0,14304.2,285600,5.008473
6,2021.0,14940.0,353300,4.228701
7,2022.0,10360.0,395900,2.616822


In [596]:
fig = make_subplots(
    rows=1, cols=2, subplot_titles=("Cap Rate % using NOI", "Cap Rate % using Revenue"))
#Using NOI
trace1 = go.Bar(x = cap_fp["Year"], y=cap_fp["Cap_Rate_FP"], marker=dict(color="Blue", opacity=1), name="Fort Pike")
fig.add_trace(trace1, row=1, col=1)
trace2 = go.Bar(x = cap_bl["Year"], y=cap_bl["Cap_Rate_BL"], marker=dict(color="green", opacity=1), name="Bright Leaf")
fig.add_trace(trace2, row=1, col=1)

#Using Revenue
trace3 = go.Bar(x = cap_fp_["Year"], y=cap_fp_["Cap_Rate_FP"], marker=dict(color="Blue", opacity=1), name="Fort Pike")
fig.add_trace(trace3, row=1, col=2)
trace4 = go.Bar(x = cap_bl_["Year"], y=cap_bl_["Cap_Rate_BL"], marker=dict(color="green", opacity=1), name="Bright Leaf")
fig.add_trace(trace4, row=1, col=2)

fig.update_xaxes(title_text="Year", row = 1, col =1)
fig.update_yaxes(title_text="Cap Rate %", range=[0,20], row = 1, col = 1)
fig.update_xaxes(title_text="Year", row = 1, col =2)
fig.update_yaxes(title_text="Cap Rate %", range=[0,20], row = 1, col = 2)
fig.update_layout(title = "Cap Rate", barmode="group")

In [597]:
write_file("CapRate.html", fig.to_html())

### Look at Cash on Cash Returns (Net Operating income / Total Cash Investment)

In [598]:
coc_fp = noi_fp.copy()
coc_fp["Cash_Invest"] = prop.loc[prop["Property"]=="Fort Pike"]["Purchase Price"].iloc[0]
coc_fp["COC"] = (coc_fp["Amount_FP"]/coc_fp["Cash_Invest"])*100

In [599]:
coc_fp

,Year,Amount_FP,Cash_Invest,COC
0,2015,13607.18,187000,7.276567
1,2016,12849.92,187000,6.871615
2,2017,13452.89,187000,7.194059
3,2018,10203.81,187000,5.456583
4,2019,15145.52,187000,8.099209
5,2020,10837.02,187000,5.795198
6,2021,11216.79,187000,5.998283
7,2022,8593.19,187000,4.595289


In [600]:
coc_bl = noi_bl.copy()
coc_bl["Cash_Invest"] = prop.loc[prop["Property"]=="Bright Leaf"]["Purchase Price"].iloc[0]
coc_bl["COC"] = (coc_bl["Amount_BL"]/coc_bl["Cash_Invest"])*100

In [601]:
coc_bl

,Year,Amount_BL,Cash_Invest,COC
0,2015,9182.31,109000,8.424138
1,2016,9594.04,109000,8.801872
2,2017,6906.62,109000,6.336349
3,2018,9605.15,109000,8.812064
4,2019,7776.90,109000,7.134771
5,2020,9515.75,109000,8.730046
6,2021,12453.00,109000,11.424771
7,2022,7401.30,109000,6.790183


In [602]:
fig = make_subplots(
    rows=1, cols=1)
#Fort Pike Graph
trace1 = go.Bar(x = coc_fp["Year"], y=coc_fp["COC"], marker=dict(color="Blue", opacity=1), name="Fort Pike")
fig.add_trace(trace1)

#Bright Leaf Graph
trace2 = go.Bar(x = coc_bl["Year"], y=coc_bl["COC"], marker=dict(color="green", opacity=1), name="Bright Leaf")
fig.add_trace(trace2)

fig.update_xaxes(title_text="Year")
fig.update_yaxes(title_text="Cash on Cash Returns")
fig.update_layout(title = "Cash on Cash", barmode="group")

In [603]:
write_file("COC.html", fig.to_html())

In [458]:
def write_file(path, html_string):
    with open(path, 'w') as f:
        f.write(html_string)
        f.close()